### chroma 내부 db 생성

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
document_list[52]

In [ ]:
from langchain_ollama.embeddings import OllamaEmbeddings

#before > terminal command: ollama pull bge-m3
embeddings = OllamaEmbeddings(model="bge-m3") #nomic-embed-text

In [ ]:
from langchain_chroma import Chroma

database = Chroma.from_documents(
    documents=document_list,
    embedding=embeddings,
    persist_directory="./chroma-markdown-tax-bge-m3",
    collection_name="chroma_markdown_tax_bge_m3",
)

In [ ]:
print(f"총 chunk 수: {database._collection.count()}")

In [ ]:
query = "원천징수 영수증은 언제 발급받을 수 있나요?" #"연간 소득이 5천만원인 직장인의 평균 종합 소득세 계산해줘."

In [ ]:
retriever = database.as_retriever(search_kwargs={"k": 4})
retriever.invoke(query)

In [ ]:
docs = database.similarity_search_with_score(query, k=4)
for doc, score in docs:
    print(f"score: {score}")
    print(f"content: {doc.page_content[:100]}")
    print("---")